In [1]:
import s3fs
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import re
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
import fiona

In [2]:
# Initialize S3 filesystem
fs = s3fs.S3FileSystem(anon=True)

# Define today and yesterday
today = datetime(2025,5,28) # datetime.now().date()
yesterday = today - timedelta(days=1)

# Build candidate folder names
dates_to_try = [today, yesterday]
forecast_folders = [f"nwm.{d.strftime('%Y%m%d')}" for d in dates_to_try]

In [3]:
print(dates_to_try)
print(forecast_folders)

[datetime.datetime(2025, 5, 28, 0, 0), datetime.datetime(2025, 5, 27, 0, 0)]
['nwm.20250528', 'nwm.20250527']


# To export as h5 file

In [8]:
idlist = [5781223, 5781221, 5781703]
cdstart = datetime.now()

for m in forecast_folders: # For the specified 2-day period 
    for i in range(0,24): # For the 24-hour period 
        i = str(i).zfill(2)
        # This generates a list of strings with filenames
        s3path = 's3://noaa-nwm-pds/' + m + '/short_range/nwm.t' + i + 'z.short_range.channel_rt*'
        remote_files = fs.glob(s3path)
        # print(len(remote_files))
        # print(len(remote_files)/18 == 24)
        # print(remote_files)

        fileset = [fs.open(file) for file in remote_files]
        # print(len(fileset))
        data = xr.open_mfdataset(fileset, combine='by_coords', parallel = True)
        print(data.model_initialization_time)
        data.close()

        for j in ['streamflow', 'qSfcLatRunoff', 'qBucket']: 
            df = data[j].sel(feature_id=idlist).to_dataframe()
            # Reset Index
            df1 = df.reset_index()[['time', 'feature_id', j]]
            # Group by feature_id and set time as index
            grouped = df1.groupby('feature_id')
            # Build a new dataframe by aligning all groups on 'time'
            df_list = []
            for feature_id, group in grouped:
                group = group.set_index('time')
                series = group[j]
                series.name = feature_id
                df_list.append(series)
            # Concatenate along columns
            df_wide = pd.concat(df_list, axis=1)
            df_wide = df_wide.rename_axis('feature_id', axis=1)
            df_wide.index = df_wide.index.strftime('%Y-%m-%d %H:%M:%S+00:00')
            df_wide
            # Export to h5 file 
            if j == 'streamflow': 
                df_wide.to_hdf('../../' + m + i + '.h5', key='short_range__' + j)
            else: 
                df_wide.to_hdf('../../' + m + i + '.h5', key='short_range__' + j, mode='a')

cdend = datetime.now()
print(f"the code took {cdend-cdstart} to run")

['noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f002.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f003.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f004.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f005.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f006.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f007.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f008.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f009.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f010.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f011.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm

# To create data frame

In [5]:
idlist = [5781223, 5781221, 5781703]
df_list = []
cdstart = datetime.now()

for m in forecast_folders: # For the specified 2-day period 
    # s3path = 's3://noaa-nwm-pds/' + m + '/short_range/*.channel_rt*' # grabs all 24 hours at once 
    for i in range(0,24): # For the 24-hour period 
        i = str(i).zfill(2)
        # This generates a list of strings with filenames
        s3path = 's3://noaa-nwm-pds/' + m + '/short_range/nwm.t' + i + 'z.short_range.channel_rt*'
        remote_files = fs.glob(s3path)
        # print(len(remote_files))
        # print(len(remote_files)/18 == 24)
        # print(remote_files)

        fileset = [fs.open(file) for file in remote_files]
        # print(len(fileset))
        data = xr.open_mfdataset(fileset, combine='by_coords', parallel = True)
        print(data.model_initialization_time)
        data.close()

        # Change to DataFrame 
        df_list.append(data.sel(feature_id=idlist).to_dataframe())

print(df_list)
cdend = datetime.now()
print(f"the code took {cdend-cdstart} to run")

2025-05-28_00:00:00
2025-05-28_01:00:00
2025-05-28_02:00:00
2025-05-28_03:00:00
2025-05-28_04:00:00
2025-05-28_05:00:00
2025-05-28_06:00:00
2025-05-28_07:00:00
2025-05-28_08:00:00
2025-05-28_09:00:00
2025-05-28_10:00:00
2025-05-28_11:00:00
2025-05-28_12:00:00
2025-05-28_13:00:00
2025-05-28_14:00:00
2025-05-28_15:00:00
2025-05-28_16:00:00
2025-05-28_17:00:00
2025-05-28_18:00:00
2025-05-28_19:00:00
2025-05-28_20:00:00
2025-05-28_21:00:00
2025-05-28_22:00:00
2025-05-28_23:00:00
2025-05-27_00:00:00
2025-05-27_01:00:00
2025-05-27_02:00:00
2025-05-27_03:00:00
2025-05-27_04:00:00
2025-05-27_05:00:00
2025-05-27_06:00:00
2025-05-27_07:00:00
2025-05-27_08:00:00
2025-05-27_09:00:00
2025-05-27_10:00:00
2025-05-27_11:00:00
2025-05-27_12:00:00
2025-05-27_13:00:00
2025-05-27_14:00:00
2025-05-27_15:00:00
2025-05-27_16:00:00
2025-05-27_17:00:00
2025-05-27_18:00:00
2025-05-27_19:00:00
2025-05-27_20:00:00
2025-05-27_21:00:00
2025-05-27_22:00:00
2025-05-27_23:00:00
[                                       

In [6]:
pd.concat(df_list)

crs  streamflow  nudge  \
time                feature_id reference_time                                
2025-05-28 01:00:00 5781223    2025-05-28 00:00:00  b''        0.05  -0.01   
                    5781221    2025-05-28 00:00:00  b''        0.05   0.00   
                    5781703    2025-05-28 00:00:00  b''        0.34   0.01   
2025-05-28 02:00:00 5781223    2025-05-28 00:00:00  b''        0.06  -0.01   
                    5781221    2025-05-28 00:00:00  b''        0.05   0.00   
...                                                 ...         ...    ...   
2025-05-28 16:00:00 5781221    2025-05-27 23:00:00  b''        0.12   0.00   
                    5781703    2025-05-27 23:00:00  b''        0.71   0.00   
2025-05-28 17:00:00 5781223    2025-05-27 23:00:00  b''        0.53   0.00   
                    5781221    2025-05-27 23:00:00  b''        0.15   0.00   
                    5781703    2025-05-27 23:00:00  b''        0.72   0.00   

                                                    velocity  qSfcLatRunoff  \
time                feature_id reference_time                                 
2025-05-28 01:00:00 5781223    2025-05-28 00:00:00      0.21        0.34690   
                    5781221    2025-05-28 00:00:00      0.26        0.00000   
                    5781703    2025-05-28 00:00:00      0.40        0.00000   
2025-05-28 02:00:00 5781223    2025-05-28 00:00:00      0.23        0.35215   
                    5781221    2025-05-28 00:00:00      0.26        0.00000   
...                                                      ...            ...   
2025-05-28 16:00:00 5781221    2025-05-27 23:00:00      0.37        0.07288   
                    5781703    2025-05-27 23:00:00      0.52        0.00000   
2025-05-28 17:00:00 5781223    2025-05-27 23:00:00      0.47        0.36916   
                    5781221    2025-05-27 23:00:00      0.40        0.10781   
                    5781703    2025-05-27 23:00:00      0.52        0.00000   

                                                    qBucket  qBtmVertRunoff  
time                feature_id reference_time                                
2025-05-28 01:00:00 5781223    2025-05-28 00:00:00  0.15686      711.800034  
                    5781221    2025-05-28 00:00:00  0.04790      248.179012  
                    5781703    2025-05-28 00:00:00  0.06723      265.647013  
2025-05-28 02:00:00 5781223    2025-05-28 00:00:00  0.15696      713.563034  
                    5781221    2025-05-28 00:00:00  0.04796      249.403012  
...                                                     ...             ...  
2025-05-28 16:00:00 5781221    2025-05-27 23:00:00  0.04894      264.466013  
                    5781703    2025-05-27 23:00:00  0.06758      277.335013  
2025-05-28 17:00:00 5781223    2025-05-27 23:00:00  0.15847      710.632034  
                    5781221    2025-05-27 23:00:00  0.04901      264.002013  
                    5781703    2025-05-27 23:00:00  0.06760      277.012013  

[2592 rows x 7 columns]

# IGNORE ME! TEST material after this 

In [66]:
# Find the latest valid short_range/channel_rt folder
valid_folder = None
for folder in forecast_folders:
    sr_path = f"noaa-nwm-pds/{folder}/short_range"
    try:
        files = fs.ls(sr_path)
        if any(".channel_rt." in f for f in files):
            valid_folder = sr_path
            break
    except Exception:
        continue

if not valid_folder:
    raise RuntimeError("No forecast folder with channel_rt found for today/yesterday")

print(valid_folder)

noaa-nwm-pds/nwm.20250528/short_range


In [5]:
# Select the forecast hour files of interest
forecast_hours = list(range(1, 19))  # Adjust as needed
selected_files = [
    f for f in files if (
        f.endswith(".nc") and 
        ".channel_rt." in f and 
        any(f"f{h:03d}" in f for h in forecast_hours)
    )
]
print(forecast_hours)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [33]:
# Extract streamflow for a given reach_id
reach_id = 1631525  # Replace or loop through a list if needed
all_data = []
print(selected_files)

['noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f002.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f003.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f004.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f005.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f006.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f007.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f008.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f009.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f010.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm.t00z.short_range.channel_rt.f011.conus.nc', 'noaa-nwm-pds/nwm.20250702/short_range/nwm

In [67]:
# This generates a list of strings with filenames
s3path = 's3://noaa-nwm-retrospective-3-0-pds/CONUS/nwm.20250528/netcdf/CHRTOUT/2025/nwm.t00z.short_range.channel_rt*'
remote_files = fs.glob(s3path)
print(remote_files)
print(len(remote_files))

[]
0


In [ ]:
# This generates a list of strings with filenames
s3path = 's3://noaa-nwm-pds/nwm.20250528/short_range/*.channel_rt*' #nwm.t00z.short_range.channel_rt*'
remote_files = fs.glob(s3path)
# print(remote_files)
print(len(remote_files))

In [102]:
# This generates a list of strings with filenames
s3path = 's3://noaa-nwm-pds/nwm.20250528/short_range/*.channel_rt*' #nwm.t00z.short_range.channel_rt*'
remote_files = fs.glob(s3path)
# print(remote_files)
print(len(remote_files))
print(len(remote_files)/18 == 24)
print(remote_files)

432
True
['noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f002.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f003.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f004.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f005.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f006.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f007.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f008.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f009.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f010.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f011.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_

['noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f002.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f003.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f004.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f005.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f006.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f007.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f008.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f009.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f010.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm.t00z.short_range.channel_rt.f011.conus.nc', 'noaa-nwm-pds/nwm.20250528/short_range/nwm

In [25]:

fileset = [fs.open(file) for file in remote_files]
print(len(fileset))

432


In [26]:
data = xr.open_mfdataset(fileset, combine='by_coords', parallel = True)
print(data)
print(data.attrs)
data.close()

<xarray.Dataset> Size: 131GB
Dimensions:         (reference_time: 24, time: 41, feature_id: 2776734)
Coordinates:
  * time            (time) datetime64[ns] 328B 2025-05-28T01:00:00 ... 2025-0...
  * feature_id      (feature_id) int64 22MB 101 179 ... 1180001803 1180001804
  * reference_time  (reference_time) datetime64[ns] 192B 2025-05-28 ... 2025-...
Data variables:
    crs             (reference_time, time) object 8kB b'' b'' b'' ... b'' b''
    streamflow      (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    nudge           (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    velocity        (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    qSfcLatRunoff   (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    qBucket         (reference_time, time, feature_id) float

In [44]:
data.data_vars

Data variables:
    crs             (reference_time, time) object 8kB b'' b'' b'' ... b'' b''
    streamflow      (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    nudge           (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    velocity        (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    qSfcLatRunoff   (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    qBucket         (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>
    qBtmVertRunoff  (reference_time, time, feature_id) float64 22GB dask.array<chunksize=(1, 1, 2776734), meta=np.ndarray>

In [50]:
df = data[['streamflow', 'qSfcLatRunoff', 'qBucket']].sel(feature_id=[5781223, 5781221, 5781703]).to_dataframe()

In [51]:
df.head()

streamflow  qSfcLatRunoff  \
reference_time time                feature_id                              
2025-05-28     2025-05-28 01:00:00 5781223           0.05        0.34690   
                                   5781221           0.05        0.00000   
                                   5781703           0.34        0.00000   
               2025-05-28 02:00:00 5781223           0.06        0.35215   
                                   5781221           0.05        0.00000   

                                               qBucket  
reference_time time                feature_id           
2025-05-28     2025-05-28 01:00:00 5781223     0.15686  
                                   5781221     0.04790  
                                   5781703     0.06723  
               2025-05-28 02:00:00 5781223     0.15696  
                                   5781221     0.04796

In [52]:
df.tail()

streamflow  qSfcLatRunoff  \
reference_time      time                feature_id                              
2025-05-28 23:00:00 2025-05-29 16:00:00 5781221           0.12        0.07188   
                                        5781703           0.70        0.04847   
                    2025-05-29 17:00:00 5781223           0.50        0.42932   
                                        5781221           0.15        0.10496   
                                        5781703           0.70        0.01861   

                                                    qBucket  
reference_time      time                feature_id           
2025-05-28 23:00:00 2025-05-29 16:00:00 5781221     0.05059  
                                        5781703     0.06829  
                    2025-05-29 17:00:00 5781223     0.16027  
                                        5781221     0.05066  
                                        5781703     0.06833

In [ ]:
df1 = df.streamflow.reset_index()[['time', 'feature_id', 'streamflow']]

# Group by feature_id and set time as index
grouped = df1.groupby('feature_id')

# Build a new dataframe by aligning all groups on 'time'
df_list = []
for feature_id, group in grouped:
    group = group.set_index('time')
    series = group['streamflow']
    series.name = feature_id
    df_list.append(series)

# Concatenate along columns
df_wide = pd.concat(df_list, axis=1)
df_wide = df_wide.rename_axis('feature_id', axis=1)
df_wide.index = df_wide.index.strftime('%Y-%m-%d %H:%M:%S+00:00')
df_wide


feature_id,5781221,5781223,5781703
time,,,
2025-05-28 01:00:00+00:00,0.05,0.05,0.34
2025-05-28 02:00:00+00:00,0.05,0.06,0.34
2025-05-28 03:00:00+00:00,0.05,0.08,0.33
2025-05-28 04:00:00+00:00,0.05,0.10,0.32
2025-05-28 05:00:00+00:00,0.05,0.14,0.32
...,...,...,...
2025-05-29 13:00:00+00:00,0.12,0.47,0.70
2025-05-29 14:00:00+00:00,0.12,0.49,0.71
2025-05-29 15:00:00+00:00,0.12,0.47,0.71


In [114]:
df_wide.to_hdf('../../test.h5', key='short_range__streamflow')

In [128]:
with h5py.File('../../test_00.h5', 'r') as f:
    # List the top-level keys (groups or datasets) in the file
    print("Keys in the HDF5 file:", list(f.keys()))
    print(len(f.keys()))

for i in ['streamflow', 'qSfcLatRunoff', 'qBucket']: 
    temp = pd.read_hdf('../../test_00.h5', key='short_range__' + i)
    # temp.loc[:,5781703] #, '5781221', '5781703']]
    print(temp.head())

Keys in the HDF5 file: ['short_range__qBucket', 'short_range__qSfcLatRunoff', 'short_range__streamflow']
3
feature_id                 5781221  5781223  5781703
time                                                
2025-05-28 01:00:00+00:00     0.05     0.05     0.34
2025-05-28 02:00:00+00:00     0.05     0.06     0.34
2025-05-28 03:00:00+00:00     0.05     0.08     0.33
2025-05-28 04:00:00+00:00     0.05     0.10     0.32
2025-05-28 05:00:00+00:00     0.05     0.14     0.32
feature_id                 5781221  5781223  5781703
time                                                
2025-05-28 01:00:00+00:00      0.0  0.34690      0.0
2025-05-28 02:00:00+00:00      0.0  0.35215      0.0
2025-05-28 03:00:00+00:00      0.0  0.31742      0.0
2025-05-28 04:00:00+00:00      0.0  0.34539      0.0
2025-05-28 05:00:00+00:00      0.0  0.36711      0.0
feature_id                 5781221  5781223  5781703
time                                                
2025-05-28 01:00:00+00:00  0.04790  0.15686  

In [30]:
import h5py

with h5py.File('../../densifimcation_shared/forcing/20220822_event/2022082100.h5', 'r') as f:
    # List the top-level keys (groups or datasets) in the file
    print("Keys in the HDF5 file:", list(f.keys()))
    print(len(f.keys()))

Keys in the HDF5 file: ['data_assimilation__qBtmVertRunoff', 'data_assimilation__qBucket', 'data_assimilation__qSfcLatRunoff', 'data_assimilation__streamflow', 'no_data_assimilation__qBucket', 'no_data_assimilation__qSfcLatRunoff', 'no_data_assimilation__streamflow', 'short_range__qBtmVertRunoff', 'short_range__qBucket', 'short_range__qSfcLatRunoff', 'short_range__streamflow']
11


In [42]:
temp = pd.read_hdf('../../densifimcation_shared/forcing/20240122_event/2024012200.h5', key='no_data_assimilation__streamflow')
# temp.loc[:,5781703] #, '5781221', '5781703']]
temp.head()

feature_id,1629509,1629511,1629513,1629515,1629517,1629521,1629523,1629525,1629527,1629529,...,5790220,5790226,5790228,5790230,5790232,5793588,24670162,24670164,24670292,24670294
time,,,,,,,,,,,,,,,,,,,,,
2024-01-22 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.129999,0.0,45.129999,45.209999,1.83,45.109999,0.0,0.44,0.19,0.0
2024-01-22 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.119999,0.0,45.119999,45.199999,1.83,45.099999,0.0,0.44,0.19,0.0
2024-01-22 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.099999,0.0,45.109999,45.189999,1.83,45.089999,0.0,0.44,0.19,0.0
2024-01-22 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.089999,0.0,45.099999,45.179999,1.83,45.079999,0.0,0.44,0.19,0.0
2024-01-22 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.069999,0.0,45.089999,45.169999,1.83,45.069999,0.0,0.44,0.19,0.0
